![banner](../logo/banner.png)

# 3.3 Entradas digitales (inputs)

Las entradas digitales son solo 2 formas, recibir un *nivel logico 0* o un *nivel logico 1*, las cuales indican un estado en la entrada y con base a ello tomar desiciones para realizar una o varias tareas.

Las entradas en el microcontrolador son `nivel lógico 1` o `alto`, lo que tendremos en pin a la entrada será un nivel de voltaje de `5V`, en caso de recibir un `0` o `bajo`, es equivalente a `0V`.


Programacion|Nivel lógico|Digital|Voltaje
:-:|:-:|:-:|:-:
False|0|Bajo|0V
True|1|Alto|5V

## 3.3.1 Recibiendo datos a la entrada

Para poder leer datos de la tarjeta debemos habilitar unas opciones más, de lo contrario no podrá leer saber si ha recibido informacion.

Debemos configuar que será entrada el *pin* y ademas activar un iterador en la libreria, el cual hace posible leer datos de entrada.

```python
# importamos los elementos necesarios
from pyfirmata import Arduino, util, INPUT

PORT = 'COM#'
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
PIN = 2
board.digital[PIN].mode = INPUT 
```

---

**Ejemplo 1: Haremos un circuito que imprima el valor si el boton esta siendo presionado**

```python
from pyfirmata import Arduino, util, INPUT
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
PIN = 2
board.digital[PIN].mode = INPUT 

while True:
    boton = board.digital[PIN].read() # leemos el valor del pin
    print(boton) # mandamos a imprimir el valor por la terminal
    sleep(0.5) # damos un tiempo entre pulsaciones
```

![input 1](imgs/input.png)

## **Otra forma de escribir el programa anterior**

```python
from pyfirmata import Arduino, util
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
board.get_pin('d:2:i')

while True:
    print(board.digital[2].read())
    sleep(0.25)
```

## 3.3.2 Pull-down vs Pull-up

Existen dos formas de conexion de un boton hacia una entrada digital, una nos da un valor de un 1 por default (pull-up), mientras que la otra nos da un valor por default de 0 (pull-down)

![pull](imgs/pulls.png)

Fisicamente quedaría las siguientes maneras

![inputs](imgs/inputs.png)

## 3.3.3 Controlando LEDs con botones

**Ejemplo 1. Ahora vamos a aprende como encender un LED por medio de un boton, cuando el boton este presionado y mande un 1, se encienda el LED, en caso que no este presionado se estara recibiendo un 0, esto causa que el LED se apague.**

![led con boton](imgs/led_boton.png)

```python
# importamos los elementos necesarios
from pyfirmata import Arduino, util, INPUT
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

PIN = 2
# configuro el pin como ENTRADA
board.digital[PIN].mode = INPUT
LED = 8 # pin donde conecto el LED

while True:
    boton = board.digital[PIN].read()
    if boton:  # es equivalente a board.digital[PIN].read() == True
        board.digital[LED].write(1)
        print('boton presionado')
        sleep(0.1) # es un tiempo para estabilizar la pulsación en el boton y no saturar la conexión
    else:
        board.digital[LED].write(0)
        sleep(0.1)
        print('boton NO presionado')
```


**Ejemplo 2. Secuencia de luces por medio de botones. Se tendran 2 botones, al presionar uno se debe generar una secuencia de luces y al presionar el otro, se debe ver otra secuencia. Mientras no se presionen los botones se debe ver un parpadeo. Con 4 LEDs.**

![secuencias](imgs/secuencias.png)

```python
from pyfirmata import Arduino, util, INPUT
from time import sleep

PORT = 'COM#'
# importamos los elementos necesarios
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

BOTON1 = 2
BOTON2 = 4
# configuro el pin como ENTRADA
board.digital[BOTON1].mode = INPUT
board.digital[BOTON2].mode = INPUT
LED1 = 8 # pin donde conecto el LED
LED2 = 9 # pin donde conecto el LED
LED3 = 10 # pin donde conecto el LED
LED4 = 11 # pin donde conecto el LED

while True:
    secuencia1 = board.digital[BOTON1].read()
    secuencia2 = board.digital[BOTON2].read()

    if secuencia1:  # es equivalente a board.digital[PIN].read() == True
        print('secuancia 1')
        board.digital[LED1].write(1)
        sleep(1)
        board.digital[LED2].write(1)
        sleep(1)
        board.digital[LED3].write(1)
        sleep(1)
        board.digital[LED4].write(1)
        sleep(1)
        board.digital[LED1].write(0)
        sleep(1)
        board.digital[LED2].write(0)
        sleep(1)
        board.digital[LED3].write(0)
        sleep(1)
        board.digital[LED4].write(0)
    elif secuencia2:
        print('secuencia2')
        board.digital[LED1].write(1)
        board.digital[LED4].write(1)
        sleep(2)
        board.digital[LED2].write(1)
        board.digital[LED3].write(1)
        sleep(2)
        board.digital[LED1].write(0)
        board.digital[LED4].write(0)
        sleep(2)
        board.digital[LED2].write(0)
        board.digital[LED3].write(0)
        sleep(2)
    else:
        print('secuancia default')
        board.digital[LED1].write(1)
        board.digital[LED2].write(1)
        board.digital[LED3].write(1)
        board.digital[LED4].write(1)
        sleep(1)
        board.digital[LED1].write(0)
        board.digital[LED2].write(0)
        board.digital[LED3].write(0)
        board.digital[LED4].write(0)
        sleep(1)
```


**Ejemplo 3. Al presionar el boton se debe quedar encendido y al volverlo a presionar debe apagarse.**

![led con boton](imgs/led_boton.png)

```python
from pyfirmata import Arduino, util, INPUT
from time import sleep

PORT = 'COM#'

board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
BOTON = 2
board.digital[BOTON].mode = INPUT

LED = 8
status = False                   #El estado del LED
board.digital[LED].write(status) #El led comienza apagado

while True:
    boton = board.digital[BOTON].read()

    if boton:
        status = not status   #invierto el estado, si estaba apagado se enciende, y visiversa
        board.digital[LED].write(status)
        sleep(0.25)          # tiempo para evitar rebotes o transitorios
```

**Ejemplo 3. Se desea crear un contador basico con un display de 7 segmentos, el cambio se hara con un boton. El contador va de 0 al 9. Cada vez que se presione el boton el valor va incrementando, cuando llegue a 9, el contador se reinicia a 0**

![display boton](imgs/btn_display.png)

```python
from pyfirmata import Arduino, util, INPUT
from time import sleep

def display(number):
    numbers = [number_0,number_1,number_2,number_3,number_4,number_5,number_6,number_7,number_8,number_9]

    for position,segments in enumerate(numbers):
        if position == number:
            segments()
            break

def number_0():
    set_segment(1,1,1,1,1,1,0)

def number_1():
    set_segment(0,1,1,0,0,0,0)

def number_2():
    set_segment(1,1,0,1,1,0,1)

def number_3():
    set_segment(1,1,1,1,0,0,1)

def number_4():
    set_segment(0,1,1,0,0,1,1)

def number_5():
    set_segment(1,0,1,1,0,1,1)

def number_6():
    set_segment(0,0,1,1,1,1,1)

def number_7():
    set_segment(1,1,1,0,0,0,0)

def number_8():
    set_segment(1,1,1,1,1,1,1)

def number_9():
    set_segment(1,1,1,1,0,1,1)

def set_segment(*segments):
    display_pins = [2,3,4,5,6,7,8]

    for pin, segment in enumerate(segments):
        print(f'Pin a encender{board.digital[display_pins[pin]]}')
        board.digital[display_pins[pin]].write(segment)

PORT = '/tmp/ttyS1'
# importamos los elementos necesarios
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
BOTON = 12
board.digital[BOTON].mode = INPUT

count = 0
display(count)
sleep(0.1)

while True:
    boton = board.digital[BOTON].read()

    if boton:
        count += 1
        sleep(0.25)
        display(count)

        if count == 9: count = -1
```

---

**Forma fea**

```python
from pyfirmata import Arduino, util, INPUT
from time import sleep


def display7(A,B,C,D,E,F,G,board,time=1 ):

    # segmentos del display
    SEGMENT_A = 2
    SEGMENT_B = 3
    SEGMENT_C = 4
    SEGMENT_D = 5
    SEGMENT_E = 6
    SEGMENT_F = 7
    SEGMENT_G = 8

    board.digital[SEGMENT_A].write(A)
    board.digital[SEGMENT_B].write(B)
    board.digital[SEGMENT_C].write(C)
    board.digital[SEGMENT_D].write(D)
    board.digital[SEGMENT_E].write(E)
    board.digital[SEGMENT_F].write(F)
    board.digital[SEGMENT_G].write(G)

    sleep(time)


PORT = 'COM#'
# importamos los elementos necesarios
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
BOTON = 12
board.digital[BOTON].mode = INPUT

count = 0 # variable me ayuda a saber en la posicion en la que estoy
sleep(0.1)

display7(1,1,1,1,1,1,0,board) # coloco el cero

while True:
    boton = board.digital[BOTON].read()

    if boton:
        count += 1

        if count >= 5: # reset, el reinicio del conteo
            count =0

        if count == 0:
            print('muestro el 0')
            display7(1,1,1,1,1,1,0,board=board)
        elif count == 1:
            print('muestro el 1')
            display7(0,1,1,0,0,0,0,board=board)
        elif count == 2:
            print('muestro el 2')
            display7(1,1,0,1,1,0,1,board=board)
        elif count == 3:
            print('muestro el 3')
            display7(1,1,1,1,0,0,1,board=board)
        elif count == 4:
            print('muestro el 4')
            display7(1,0,0,0,1,1,1,board=board)
```

![display btn simulacion](imgs/display_btn_simul.png)

**Ejemplo 4. Controlar la inversion de giro de un motor DC, con un boton el motor gira en un sentido y se queda funcionado hasta que se presione el mismo boton. Con otro boton gira en sentido contrario y si lo vuelvo a presionar que se apague el motor. En caso de presionar el boton contrario el motor se debe apagar un instate e invertir el giro.**

```python
from pyfirmata import Arduino, util
from time import sleep

def giro(board, pins=[2,3],values=[0,0]):
    board.digital[pins[0]].write(values[0])
    board.digital[pins[1]].write(values[1])
    sleep(0.5) # estabilizar la salida al motor


PORT = 'COM#'
# importamos los elementos necesarios
board = Arduino(PORT)

it = util.Iterator(board) # le pasamos la tarjeta al iterador
it.start() #arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
BOTON_R = 8
BOTON_L = 9

board.get_pin(f'd:{str(BOTON_R)}:i')
board.get_pin(f'd:{str(BOTON_L)}:i')

# Configuro los pines del motor
MOTOR_1 = 2
MOTOR_2 = 3

# variables que me indican el estado del motor y su sentido
status_r = False
status_l = False

while True:

    boton_r = board.digital[BOTON_R].read()
    boton_l = board.digital[BOTON_L].read()

    sleep(0.02) #estabilizar la lectura del boton

    if boton_r:
        if not status_r and not status_l:
            giro(board, [MOTOR_1, MOTOR_2], [1, 0])
            status_r = True
        elif status_l:
            giro(board, [MOTOR_1, MOTOR_2], [0, 0])
            sleep(0.25)
            giro(board, [MOTOR_1, MOTOR_2], [1, 0])
            status_r = True
            status_l = False
        else:
            giro(board, [MOTOR_1, MOTOR_2], [0, 0])
            status_r = False

    if boton_l:
        if not status_l and not status_r:
            giro(board, [MOTOR_1, MOTOR_2], [0, 1])
            status_l = True
        elif status_r:
            giro(board, [MOTOR_1, MOTOR_2], [0, 0])
            sleep(0.25)
            giro(board, [MOTOR_1, MOTOR_2], [0, 1])
            status_l = True
            status_r = False
        else:
            giro(board, [MOTOR_1, MOTOR_2], [0, 0])
            status_l = False
```

![motor simulacion](imgs/motor_btn_simul.png)

## Modo de configuracion de pines `get_pin('t:p:m')`

Existe otra manera de configurar los pines, es con el método `get_pin()`, el valor se pasa como string `str` separados por dos puntos `:`.

**Tipo**

- `a` : analogico
- `d` : digital

**Número de pin en la tarjeta:**

- `pin`: del 2 al 13
    
**Modo:**

- `i`: Entrada (input)
- `o`: Salida (output)
- `p`: PWM (salida analogica)

**Ejemplos:**

```python
# pin 2 (D2) como salida de digital
pin2 = board.get_pin('d:2:o')
pin2.write(1)

# pin 4 (D4) como entrada de digital
pin4 = board.get_pin('d:4:i')
pin4.write(1)

# pin 0 (A0) como entrada analogica
analog_0 = board.get_pin('a:0:i')
analog_0.read()

# pin 3 (D3) como salida en PWM (Salida analogica)
pin3 = board.get_pin('d:3:p')
pin3.write(0.6)
```

## Tarea - Entradas digitales:

1. Realizar 3 secuencias de luces, cada secuencia se activa al presionar su boton correspondiente. Cuando no se esta presionando ningun boton debe encender y apagarse en secuencia los LEDs, es decir, enciende el primero, despues el segundo, hacia hasta encender todos, y se apagan en el mismo orden, del ultimo al primero.
2. Crear un contador con un display de 7 segmentos. Tendra 3 botones, uno de incremento, el segundo de decremento y el ultimo de reset. 
3. Vamos a controlar un motor DC, se tienen 2 botones cuando se presione el primero el motor girar a la derecha y cuando se presione el segundo debe girar a la izquierda, se deben agregar un led indicativo para cada sentido. Para detener el motor se debe presionar el mismo boton que se presiono previo a que girara.
4. Tenemos 2 motores DC, cada uno se debe controlar con sus propios botones que a su vez controlan el sentido del giro. Vamos a tener 4 botones, 2 para cada motor y a su vez cada uno es para su giro a la derecha e izquierda. Ambos motores pueden estar funcionando al mismo tiempo. Para detener el motor se debe presionar el mismo boton que se presiono previo a que girara.

---

Realizado por Docente: [Alejandro Leyva](https://www.alejandro-leyva.com/)

[Mecatrónica 85](https://mecatronica85.com/)

[fb/mecatronica85](https://www.facebook.com/mecatronica85)